In [43]:
from matplotlib import pyplot as plt
import pandas as pd
import MeCab
import glob
from sklearn.feature_extraction.text import TfidfVectorizer

def twoeet_to_wordcloud(col = 'Profile',filename = '../TweetData/Tweet_報道ステーションOR報ステORジェンダー平等OR#報道ステーションOR$報ステ.csv'):
    tmp = pd.read_csv(filename)
    tmp[col] = tmp[col].map(str)
    tmp = tmp[tmp[col].isnull() == False]
    mikudashi = '馬鹿|アホ|バカ|クズ|呆れる|無能|クソ|能無し|キチガイ|低能|恥知らず|卑怯|間抜け|愚か|下衆|腰抜け|幼稚|偽善者|厚顔無恥|情弱|底辺層|無教養|乞食|マスゴミ|シナチョン|井の中の蛙|老害|ガイジ|池沼|無能|害悪|キモオタ|貧乏人|売国奴|愚か者|腰抜け|ノータリン|滑稽|国賊|政治屋|下劣|人非人|愚鈍|気狂い|マヌケ|愚劣|痴れ者|しょうもない|アンポンタン|嘘つき|生意気|浅はか|自業自得|無神経|軽蔑|情報弱者|ド低脳|莫迦|キモい|穀潰し|恥晒し|嘘吐き|無様'
    tmp_mikudashi = tmp[tmp['contents'].str.contains(mikudashi)]
    tmp_not_mikudashi = tmp[~tmp['contents'].str.contains(mikudashi)]
    tmp_mikudashi = tmp_mikudashi[~tmp_mikudashi.duplicated(subset = 'Screen_name')]
    tmp_not_mikudashi = tmp_not_mikudashi[~tmp_not_mikudashi.duplicated(subset = 'Screen_name')]

    """tmplist = tmp_not_mikudashi['contents'].str.replace('(http[:a-zA-Z./]*)|@[a-zA-Z_]*|RT','',regex=True).tolist()
    tmplist_mikudashi = tmp_mikudashi['contents'].str.replace('(http[:a-zA-Z./]*)|@[a-zA-Z_]*|RT','',regex=True).tolist()"""
    tmplist = tmp_not_mikudashi[col].str.replace('[^ぁ-んァ-ンー一-龥]','',regex=True).tolist()
    tmplist_mikudashi = tmp_mikudashi[col].str.replace('[^ぁ-んァ-ンー一-龥]','',regex=True).tolist()
    result = [tmplist, tmplist_mikudashi]
    return(result)

# Taggerオブジェクトを生成

def extract(text):
    tokenizer = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd') 
    tokenizer.parse("")
    stop_words_ja = ['もの', 'こと', 'とき', 'そう', 'たち', 'これ', 'よう', 'これら', 'それ', 'すべて',
     'の', 'さん', '人', 'ん','笑','今日','円','アカウント','垢','好き','フォロー','事','大好き','趣味',
     '最近','時','無言','失礼','年','ツイート','自分','中']
    txtlen = 0
    text_result_list = []
    while len(text) > txtlen - 1000:
        tmp_text = ''.join(text[txtlen:txtlen+1000])
        words = []
        # 単語の特徴リストを生成
        node = tokenizer.parseToNode(tmp_text)
        while node:
            # 品詞情報(node.feature)が名詞ならば
            #if node.feature.split(",")[0] == u"名詞":
                # 単語(node.surface)をwordsに追加
                #words.append(node.surface)
            word_type = node.feature.split(',')[0]
            word_surf = node.surface.split(',')[0]
            if word_type == '名詞' and word_surf not in stop_words_ja:
                words.append(node.surface)
            #words.append(node.surface)
            node = node.next
        # 半角スペース区切りで文字列を結合
        text_result = ' '.join(words)
        text_result_list.append(text_result)
        txtlen += 1000
    text_result = ' '.join(text_result_list)
    return text_result

def tfidf_extract(col = 'Profile',filename = '../TweetData/Tweet_報道ステーションOR報ステORジェンダー平等OR#報道ステーションOR$報ステ.csv'):
    textlist = twoeet_to_wordcloud(col,filename)
    docs = []
    for prf in textlist:
        text = extract(prf)
        docs.append(text)
    vectorizer = TfidfVectorizer(smooth_idf=False)
    X = vectorizer.fit_transform(docs)

    # データフレームに表現
    values = X.toarray()
    feature_names = vectorizer.get_feature_names()
    df = pd.DataFrame(values, columns = feature_names, index=['普通','見下し'])
    df = df.T
    comlist = []
    for column in df.columns:
        comlist.append(df.sort_values(column,ascending = False).index[0:30])
    comlist = [list(com) for com in comlist]
    print('filename:'+filename)
    print('普通ユーザ:')
    print(comlist[0])
    print('\n')
    print('見下しユーザ:')
    print(comlist[1])
    print('\n')

import glob
from multiprocessing import Process
mikudashi = '馬鹿|アホ|バカ|クズ|呆れる|無能|クソ|能無し|キチガイ|低能|恥知らず|卑怯|間抜け|愚か|下衆|腰抜け|幼稚|偽善者|厚顔無恥|情弱|底辺層|無教養|乞食|マスゴミ|シナチョン|井の中の蛙|老害|ガイジ|池沼|無能|害悪|キモオタ|貧乏人|売国奴|愚か者|腰抜け|ノータリン|滑稽|国賊|政治屋|下劣|人非人|愚鈍|気狂い|マヌケ|愚劣|痴れ者|しょうもない|アンポンタン|嘘つき|生意気|浅はか|自業自得|無神経|軽蔑|情報弱者|ド低脳|莫迦|キモい|穀潰し|恥晒し|嘘吐き|無様'
filenamelist = glob.glob('../TweetData/Tweet_*.csv')
process_list = []
"""for filename in filenamelist:
    process = Process(
        target=tfidf_extract,
        kwargs={
            'col': 'Profile',
            'filename': filename
             })
    process.start()
    process_list.append(process)

for process in process_list:
    process.join()"""

"for filename in filenamelist:\n    process = Process(\n        target=tfidf_extract,\n        kwargs={\n            'col': 'Profile',\n            'filename': filename\n             })\n    process.start()\n    process_list.append(process)\n\nfor process in process_list:\n    process.join()"

In [46]:
tfidf_extract(filename = filenamelist[4])

filename:../TweetData/Tweet_サッカー.csv
普通ユーザ:
['ゲーム', 'サッカー', '応援', 'ちゃん', '音楽', 'アニメ', '気軽', 'くん', '野球', 'お願い', 'ファン', 'よろしくお願いします', 'アイコン', '成人', 'フォロバ', '日常', '基本', 'たま', '映画', '多め', '漫画', '東京', '仕事', '写真', '競馬', '在住', '選手', '中心', 'スポーツ', '人生']


見下しユーザ:
['サッカー', 'ゲーム', '応援', '野球', 'アニメ', '音楽', '競馬', '日本', 'ちゃん', 'アイコン', 'ファン', 'お願い', '気軽', '基本', 'フォロバ', 'スポーツ', 'よろしくお願いします', 'たま', '漫画', '人生', '東京', '成人', '選手', '観戦', 'ため', '仕事', '現在', '在住', 'くん', '映画']




In [44]:
filenamelist

['../TweetData/Tweet_車いすOR車椅子OR伊是名OR乗車拒否.csv',
 '../TweetData/Tweet_バッハORチャイニーズORIOC.csv',
 '../TweetData/Tweet_トイレットペーパーORティッシュOR生理用品OR国民生活安定緊急措置法.csv',
 '../TweetData/Tweet_オリンピックORパラリンピックOR五輪OR聖火.csv',
 '../TweetData/Tweet_サッカー.csv',
 '../TweetData/Tweet_野球.csv',
 '../TweetData/Tweet_感染者.csv',
 '../TweetData/Tweet_サントリーOR新浪OR新浪OR定年.csv',
 '../TweetData/Tweet_トリチウムOR汚染水OR処理水.csv',
 '../TweetData/Tweet_緊急事態宣言.csv',
 '../TweetData/Tweet_メダルかじりOR名古屋市長OR河村市長OR後藤希友ORソフトボール.csv',
 '../TweetData/Tweet_報道ステーションOR報ステORジェンダー平等OR#報道ステーションOR$報ステ.csv',
 '../TweetData/Tweet_開会式.csv',
 '../TweetData/Tweet_西村大臣OR経済再生相OR金融機関OR酒類.csv']

In [38]:
df = pd.read_csv(filenamelist[3])
df = df[df['Profile'].isnull() == False]
df = df[~df.duplicated(subset = 'Screen_name')]
#tmp = df[['Screen_name','Profile']][df['Profile'].str.contains('((腐)*(成人))|((成人)*(腐))')]
tmp = df[['Screen_name','Profile']][df['Profile'].str.contains('日本')]

In [37]:
filenamelist

['../TweetData/Tweet_車いすOR車椅子OR伊是名OR乗車拒否.csv',
 '../TweetData/Tweet_バッハORチャイニーズORIOC.csv',
 '../TweetData/Tweet_トイレットペーパーORティッシュOR生理用品OR国民生活安定緊急措置法.csv',
 '../TweetData/Tweet_オリンピックORパラリンピックOR五輪OR聖火.csv',
 '../TweetData/Tweet_サッカー.csv',
 '../TweetData/Tweet_野球.csv',
 '../TweetData/Tweet_感染者.csv',
 '../TweetData/Tweet_トリチウムOR汚染水OR処理水.csv',
 '../TweetData/Tweet_緊急事態宣言.csv',
 '../TweetData/Tweet_メダルかじりOR名古屋市長OR河村市長OR後藤希友ORソフトボール.csv',
 '../TweetData/Tweet_報道ステーションOR報ステORジェンダー平等OR#報道ステーションOR$報ステ.csv',
 '../TweetData/Tweet_開会式.csv',
 '../TweetData/Tweet_西村大臣OR経済再生相OR金融機関OR酒類.csv']

In [39]:
tmp

,Screen_name,Profile
9,BSCj5r2yJYGM7gT,@PRIN900凍結されました。言論の自由はどこにいった⁉︎ アメリカ大統領選挙や中国につい...
25,korpio,反市場原理主義。反新自由主義。反自由貿易 。反緊縮。反ネオコン戦争屋。反国際金融資本。反ポリ...
41,9YywJzjKCkHWXXh,皇国日本ヲ護ルベシ
44,tatamimis,アトピー重症化で只今入院中。反グローバル派の生粋の日本人。海外好きでしたが情報戦争中、改めて...
76,YugaNatsugami,政治ネタを中心に書いてます。子や孫に日本を残したい。偽善、自己中、事なかれの戦後民主主義体制...
...,...,...
349072,iroha_kabu,2018年末に退職した30代。 退職後にプログラミングの学校に通って(卒業した)、株取引を勉...
349081,altairvega7,明日死ぬつもりで生きる。 永遠に生きるつもりで学ぶ。 #音楽 #廃墟 #DJ #ボカロ ...
349089,Shibatomo_chan,80's日本のインディーズ パンク レゲエ テクノ 70〜90’S英国音楽 ゆるキャラ 映画...
349173,NekoPula,【いろいろな方の話や情報が聴けるので巻き込みリプ等大歓迎です】 巻き込まれが嫌な人はブロック...


In [40]:
df

,time,Screen_name,tweet.id,followers,friends,retweet_cnt,Profile,contents,search_word
0,2021-01-30 00:00:01,tokyo2020swim,1355168831212142594,34,241,0,東京オリンピックカウントダウンページです。競技日程、ニュース、リンクなどを掲載しています。,東京オリンピック開会式まで174日と20時間 #東京オリンピック #カウントダウン #東京2...,オリンピック OR パラリンピック OR 五輪 OR 聖火
1,2021-01-30 00:00:02,vokkhan,1355168835452588042,483,905,0,#フォロバ100 #相互フォロー,[韓国の反応]ビル・ゲイツ氏、五輪開催「可能性ある」ワクチン鍵[韓国ネット民]リップサービス...,オリンピック OR パラリンピック OR 五輪 OR 聖火
2,2021-01-30 00:00:02,interikumamon,1355168833921806342,5,0,0,世界平和と社会正義の実現が僕の夢です。 YouTubeで本を読んでいます。,毎日コンスタントに100人前後が死ぬ状況で、オリンピックを開こうとする人々。頭に蛆がワイテイ...,オリンピック OR パラリンピック OR 五輪 OR 聖火
3,2021-01-30 00:00:11,max358japan,1355168871548735488,32154,10427,0,ソースマンはブロック。しつこい巻き込みﾘﾌﾟ大嫌い！初見挨拶DMお断り。なかなか絡めないけど...,@tk1zmQ 五輪・・・終了！,オリンピック OR パラリンピック OR 五輪 OR 聖火
4,2021-01-30 00:00:13,Ubermensch_Ich,1355168880449077251,12,60,0,Lieb vaterland magst ruhig sein ...,未来予想 →東京五輪は開催される ※ただし無観客で ※あくまで個人の勝手な憶測なので、あ...,オリンピック OR パラリンピック OR 五輪 OR 聖火
...,...,...,...,...,...,...,...,...,...
349196,2021-02-09 23:59:50,hiroshima_pot,1359155049453260806,435,631,0,ツイッター始めました,北京冬季オリンピックは来年2月4日からか。2月4日にはもう出てたのかな？,オリンピック OR パラリンピック OR 五輪 OR 聖火
349197,2021-02-09 23:59:50,taqmin,1359155051604963331,981,547,0,札幌育ちの川崎在住。小杉エリマネ理事。川崎フロンターレサポ RSR FRF Jリーグ石狩支...,マジかよｗ 痛いニュース(ﾉ∀`):【悲報】 五輪公式Twitterアカウントのヘッダーがこ...,オリンピック OR パラリンピック OR 五輪 OR 聖火
349198,2021-02-09 23:59:52,jingoiitihodi,1359155056520687617,416,2263,0,クソ公式とクソ公式擁護派とアンチアンチが心底嫌いなヲタク ポリコレ振りかざす自称リベラルも...,ボランティアを多用しようとする東京オリンピック、日本の病理としか。,オリンピック OR パラリンピック OR 五輪 OR 聖火
349201,2021-02-09 23:59:55,leader_katayama,1359155069527162883,106,186,0,平凡な毎日に唄う道草節,「オリンピック開催に向けて」森会長の発言は看過できない、と「（コロナや利権を巡って）オリンピ...,オリンピック OR パラリンピック OR 五輪 OR 聖火


In [41]:
3513/56208
4679/151873
10574/151873

0.06962396212625022

In [31]:
df.contents

0         車椅子の少女、嘘つきに助けられる | 葉月２  読んでくれなきゃ幼女に騒いでグーパンされちゃ...
1         筑波大学ベンチャーシンポジウムに参加したご縁からピクシーダストテクノロジーズ社の方とオンライ...
2                 チェリー、今日も滑ったんですか？それともジョーに車椅子押してもらってたんですか？笑
3                                   やべぇNボックススラッシュ欲しい車椅子も乗るし
4         @Shiomusbi1 りんご剥こうとしたり 車椅子押したりする画が浮かんだけど どう考えて...
                                ...                        
122916    @moecha_nico_2 あごめんWWWWWWW 朝の会終わって先生に半泣きで保健室行っ...
122918                     社民党の車椅子の人、ギレルモ・デル・トロの映画に出てなかった？？
122919    今度は飲食店でトラぶって警官シバいて逮捕された方のツイート。 前半は頷ける部分もあるけど、車...
122922    「もういいよ。もう嘘吐くの、やめてよ」  【論破】 負傷者は“不運”だ。彼の身長はプロフィー...
122940                「（ふぅと息をつき車椅子から立ち上がる。それから分けていた前髪を直して）」
Name: contents, Length: 56208, dtype: object